In [1]:
# Include all the functions 
include("src/main.jl");

In [2]:
# Run all the test cases
include("tests/runtests.jl");

Test Summary:            | Pass  Total  Time
Computing CRNT structure |    6      6  0.9s
Test Summary:     | Pass  Total  Time
Checking toricity |    4      4  0.6s
Test Summary:                | Pass  Total  Time
Graph theoretic computations |    3      3  0.0s
Test Summary:                      | Pass  Total  Time
Networks with intermediate species |   11     11  0.6s
Test Summary:   | Pass  Total   Time
Shinar-Feinberg |   21     21  32.8s


In [3]:
# Matrices for the running IDH example
N = matrix(QQ, [[-1, 1, 1, 0, 0, 0],
     [-1, 1, 0, 0, 0, 1],
     [1, -1, -1, -1, 1, 1],
     [0, 0, 1, -1, 1, 0],
     [0, 0, 0, 1, -1, -1]])

M = matrix(ZZ, [[1, 0, 0, 0, 0, 0],
     [1, 0, 0, 0, 0, 0],
     [0, 1, 1, 1, 0, 0],
     [0, 0, 0, 1, 0, 0],
     [0, 0, 0, 0, 1, 1]])

[1   0   0   0   0   0]
[1   0   0   0   0   0]
[0   1   1   1   0   0]
[0   0   0   1   0   0]
[0   0   0   0   1   1]

In [4]:
n = nrows(M)
r = ncols(M)
s = rank(N)
d = n-s
println("n=$n, r=$r, d=$d")

n=5, r=6, d=2


In [5]:
# Row reduced stoichiometric matrix
C = row_space(N)

# Conserved quantities
W = kernel(N, side=:left)

[-2   1   -1   1   0]
[ 1   0    1   0   1]

In [6]:
# Graph structure and deficiency
g, embedding = reaction_graph(N, M)

# Number of complexes
m = length(Graphs.vertices(g))
println("m=$m")

# Linkage classes
ell = length(Graphs.weakly_connected_components(g))
println("ell=$ell")

# Deficiency
deficiency  = delta(N, M)
println("deficiency=$deficiency")

m=6
ell=2
deficiency=1


In [7]:
covered_by_deficiency_zero_theorem(N, M)

false

In [8]:
convered_by_deficiency_one_theorem(N, M)

false

In [9]:
# Consistency
nonempty_positive_kernel(N)

true

In [10]:
# Generic nondegeneracy
has_nondegenerate_zero(N, M)

true

In [11]:
# Intermediates
intermediates_result = intermediates(N, M, only_single_input=true)

1-element Vector{@NamedTuple{species::Int64, input_complexes::Vector{Vector{Int64}}, output_complexes::Vector{Vector{Int64}}}}:
 (species = 5, input_complexes = [[0, 0, 1, 1, 0]], output_complexes = [[0, 1, 1, 0, 0], [0, 0, 1, 1, 0]])

In [12]:
# Reduced network
Ntilde, Mtilde = reduced_network(N, M, intermediates_result)
Ctilde = row_space(Ntilde)

[1   -1   0   -1]
[0    0   1   -1]

In [13]:
# Toric invariance space
Atilde = toric_invariance_space(Ctilde, Mtilde)

[1   0   1   0]
[0   1   1   0]

In [14]:
# Lifted toric invariance space
A = lift_exponent_matrix(Atilde, M, intermediates_result)

[1   0   1   0   1]
[0   1   1   0   1]

In [15]:
# Sanity check
A_original = toric_invariance_space(C, M)
row_space(A) == row_space(A_original)

true

In [16]:
# Does the toric invariance space consist of conserved quantities?
all(is_zero, A*N)

false

In [17]:
# Injectivity test
injectivity_test(Ctilde, Mtilde, Atilde)

true

In [18]:
# Finitely many cosets for all rate constants?
all_positive_roots_nondegenerate(Ctilde, Mtilde, QQ.(Atilde))

true

In [19]:
# Mixed volume bound on the number of cosets
F = coset_counting_system(Ctilde, Mtilde, Atilde)
HC.mixed_volume(F)

2

In [20]:
# Constant number of cosets for all rate constants?
N_A_tilde = reconstruct_stoichiometric_matrix_with_row_space_and_conserved_quantities(Ctilde,QQ.(Atilde))
display(minimal_siphons(N_A_tilde, Mtilde))
siphon_test(N_A_tilde, Mtilde)

3-element Vector{Set{Int64}}:
 Set([2, 3, 1])
 Set([3, 1])
 Set([2, 3])

true

In [21]:
positive_vector_in_rowspace(Atilde)

false

In [22]:
# Number of positive solutions for random parameters
number_of_cosets_for_random_parameters(Ctilde, Mtilde, Atilde; certify=true)

1

In [23]:
# Putting it all together
coset_counting_analysis(Ntilde, Mtilde, Atilde)

Injectivity test passed for reduced network
Toricity!


(toricity = true, finite = true)

In [24]:
# Capacity for multistationarity for original network
coset_with_multistationarity(A, W)

false

In [25]:
# (Local) ACR check
zero_columns(A)

1-element Vector{Int64}:
 4

In [26]:
# Gröbner basis test for generic binomiality
F = vertical_system(C, M)
binomiality_check(F, verbose=true)

Gröbner basis: Generically binomial!
Specialization for all rate constants: Verified


(generically = true, for_all_positive = true)